<a href="https://colab.research.google.com/github/AnDDoanf/LLM-repo/blob/master/RAG_mistral7b_langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!pip install transformers bitsandbytes accelerate langchain langchain_community playwright html2text sentence-transformers faiss-gpu
!playwright install

In [2]:
%%capture
from transformers import AutoModelForCausalLM, AutoTokenizer
import transformers
from transformers import BitsAndBytesConfig
from torch import cuda, bfloat16

model_name='VietnamAIHub/Vietnamese_llama2_7B_8K_SFT_General_domain'

model_config = transformers.AutoConfig.from_pretrained(
    model_name,
)

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=bfloat16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
)

In [3]:
import requests

articles = []
for index in range(1,4):
  html = requests.get("http://triethoc.edu.vn/vi/chuyen-de-triet-hoc/triet-hoc-nhan-hoc/page/"+str(index)+"/")

  from bs4 import BeautifulSoup
  soup = BeautifulSoup(html.content, 'html.parser')

  for item in soup.find_all('div', {'class':'content-list'})[0].find_all('div', {'class':'item'}):
      articles.append("http://triethoc.edu.vn"+item.find('a')['href'])

In [4]:
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import AsyncChromiumLoader
from langchain.document_transformers import Html2TextTransformer
from langchain.vectorstores import FAISS
import nest_asyncio
from langchain.embeddings import HuggingFaceEmbeddings
nest_asyncio.apply()

# articles = ["https://printer.gotquestions.net/GeneratePF?articleId=" + str(index) for index in range(200)]

import requests
html = requests.get("http://triethoc.edu.vn/vi/chuyen-de-triet-hoc/triet-hoc-nhan-hoc/page/1/")

from bs4 import BeautifulSoup
soup = BeautifulSoup(html.content, 'html.parser')
articles = []
for item in soup.find_all('div', {'class':'content-list'})[0].find_all('div', {'class':'item'}):
    articles.append("http://triethoc.edu.vn"+item.find('a')['href'])

# Scrapes the blogs above
loader = AsyncChromiumLoader(articles)
docs = loader.load()

# Converts HTML to plain text
html2text = Html2TextTransformer()
docs_transformed = html2text.transform_documents(docs)

# Chunk text
text_splitter = CharacterTextSplitter(chunk_size=200,
                                      chunk_overlap=0)
chunked_documents = text_splitter.split_documents(docs_transformed)

# Load chunked documents into the FAISS index
db = FAISS.from_documents(chunked_documents,
                          HuggingFaceEmbeddings(model_name='sentence-transformers/all-mpnet-base-v2'))

# Connect query to FAISS index using a retriever
retriever = db.as_retriever(
    search_type="similarity",
    search_kwargs={'k': 1}
)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [5]:
from langchain.llms import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.chains import LLMChain
from langchain_core.runnables import RunnablePassthrough
text_generation_pipeline = transformers.pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    temperature=0.2,
    repetition_penalty=1.1,
    return_full_text=True,
    max_new_tokens=300,
    do_sample=True
)

prompt_template = """
### [INST]
Instruction: Answer the question based on the provided context.
If you do not know, say you don't know.
Do cite your sources.
Answer the question using Vietnamese.

Here is context to help:

{context}

### QUESTION:
{question}

[/INST]
 """



mistral_llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

# Create prompt from prompt template
prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=prompt_template,
)

# Create llm chain
llm_chain = LLMChain(llm=mistral_llm, prompt=prompt, output_key="answer")

retriever = db.as_retriever()

rag_chain = (
 {"context": retriever, "question": RunnablePassthrough()}
    | llm_chain
)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 0.3. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFacePipeline`.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


In [9]:
import torch
torch.cuda.empty_cache()

rag_chain.invoke("Dân tộc là gì?")['answer']

"\n### [INST]\nInstruction: Answer the question based on the provided context.\nIf you do not know, say you don't know.\nDo cite your sources.\nAnswer the question using Vietnamese.\n[INST]\nHere is context to help:\n\n[Document(page_content='GIẢI ĐÁP:', metadata={'source': 'http://triethoc.edu.vn/vi/chuyen-de-triet-hoc/triet-hoc-nhan-hoc/tong-luan-than-hoc-su-tao-thanh-than-the-cua-con-nguoi-dau-tien_1081.html'}), Document(page_content='GIẢI ĐÁP:', metadata={'source': 'http://triethoc.edu.vn/vi/chuyen-de-triet-hoc/triet-hoc-nhan-hoc/tong-luan-than-hoc-su-tao-thanh-than-the-cua-con-nguoi-dau-tien_1081.html'}), Document(page_content='GIẢI ĐÁP:', metadata={'source': 'http://triethoc.edu.vn/vi/chuyen-de-triet-hoc/triet-hoc-nhan-hoc/tong-luan-than-hoc-su-tao-thanh-than-the-cua-con-nguoi-dau-tien_1081.html'}), Document(page_content='* * *\\n\\n|   TIẾT 3  \\n---  \\n CÂU HỎI 93  \\n---  \\n  \\n\\n\\n### Ý KIẾN BẠN ĐỌC\\n\\n## Các tin khácXem thêm', metadata={'source': 'http://triethoc.edu.